In [1]:
import sys
from math import pi

from direct.showbase.ShowBase import ShowBase

from panda3d.core import VBase3
from panda3d.core import Vec3
from panda3d.core import Quat
from panda3d.core import invert
from panda3d.bullet import BulletWorld
from panda3d.bullet import BulletRigidBodyNode
from panda3d.bullet import BulletSphereShape
from panda3d.bullet import BulletDebugNode


# Basic setup
s = ShowBase()
s.disable_mouse()
s.accept('escape', sys.exit)
s.cam.set_pos(0, -10, 0)


# Physics
bullet_world = BulletWorld()
def run_physics(task):
    bullet_world.do_physics(globalClock.getDt())
    return task.cont
s.task_mgr.add(run_physics, sort=1)
# Debug visualization
debug_node = BulletDebugNode('Debug')
debug_node.showWireframe(True)
debug_node.showConstraints(True)
debug_node.showBoundingBoxes(False)
debug_node.showNormals(False)
debug_np = s.render.attach_new_node(debug_node)
bullet_world.set_debug_node(debug_node)
debug_np.show()


# The object in question
mass = BulletRigidBodyNode()
mass.set_mass(1)
mass.setLinearSleepThreshold(0)
mass.setAngularSleepThreshold(0)
shape = BulletSphereShape(1)
mass.add_shape(shape)
mass_node = s.render.attach_new_node(mass)
mass_node.set_hpr(0, 0, 0)
bullet_world.attach_rigid_body(mass)
model = s.loader.load_model('models/smiley')
model.reparent_to(mass_node)
model_axis = loader.load_model('models/zup-axis')
model_axis.reparent_to(model)
model_axis.set_pos(-1, -1, -1)
model_axis.set_scale(0.2)


target_node = s.loader.load_model('models/smiley')
target_node.reparent_to(s.render)
target_node.set_hpr(179, 89, 0)
target_node.set_scale(1.5)
target_node.set_render_mode_wireframe()


delta_node = s.render.attach_new_node('delta')
delta_node.set_pos(2, -1, -1)
delta_node_model = s.loader.load_model('models/smiley')
delta_node_model.reparent_to(delta_node)
delta_node_model.set_h(180)
delta_node_model.set_render_mode_wireframe()
delta_node_model.set_two_sided(True)
delta_node = s.render.attach_new_node('delta')
delta_node.set_pos(2, -1, -1)
delta_node_model = s.loader.load_model('models/smiley')
delta_node_model.reparent_to(delta_node)
delta_node_model.set_h(180)
delta_node_model.set_render_mode_wireframe()
delta_node_model.set_two_sided(True)


speed_node = s.loader.load_model('models/zup-axis')
speed_node.reparent_to(s.render)
speed_node.set_pos(2, -1, 1)
speed_node.set_hpr(10, 0, 0)


def stabilize(task):
    # inertia: kg * m^2
    # torque: N*m
    # angular_impulse: N*m*sec
    # angular_velocity: radians/sec
    # force = mass * acceleration
    # acceleration = delta_velocity / delta_time
    # impulse = force * time

    dt = globalClock.dt

    tau = 0.2
    inertia = 1.0

    orientation = mass_node.get_quat(s.render)
    angular_velocity = mass.get_angular_velocity() # radians / sec
    # speed_node.set_scale(angular_velocity / (2*pi) * 0.05 + VBase3(0.01, 0.01, 0.01))

    delta_hpr = target_node.get_hpr(mass_node)
    delta_hpr.componentwise_mult(VBase3(1, -1, 1))
    delta_node.set_hpr(delta_hpr)
    delta_angle = delta_node.get_quat().get_angle_rad()

    axis_of_torque = Vec3(
        -delta_node.get_p(s.render),
        delta_node.get_r(s.render),
        delta_node.get_h(s.render),
    )
    axis_of_torque.normalize()
    target_angular_velocity = axis_of_torque * delta_angle / tau / pi
    steering_impulse = target_angular_velocity * inertia

    # We calculate the impulse that cancels out all current rotation,
    # which is noise with regard to the intended rotation.
    countering_impulse = -angular_velocity / 2.5

    max_impulse = 0.4

    impulse = countering_impulse + steering_impulse
    if impulse.length() > max_impulse:
        impulse = impulse / impulse.length() * max_impulse

    mass.apply_torque_impulse(impulse)
    return task.cont
# s.task_mgr.add(stabilize, sort=0)


# Apply torque with 't'
def add_torque(x=0, y=0, z=0):
    # This happens in world space
    # mass.apply_torque_impulse(VBase3(x, y, z))
    mass.apply_torque(VBase3(x,y,z))
s.accept('x', add_torque, [10, 0, 0])
s.accept('y', add_torque, [0, 10, 0])
s.accept('z', add_torque, [0, 0, 10])


s.run()

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


SystemExit: 

/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
#!/usr/bin/env python

import sys

from direct.showbase.ShowBase import ShowBase
from direct.showbase.InputStateGlobal import inputState

from panda3d.core import AmbientLight
from panda3d.core import DirectionalLight
from panda3d.core import LPoint3
from panda3d.core import TransformState
from panda3d.core import BitMask32

from panda3d.bullet import BulletWorld
from panda3d.bullet import BulletBoxShape
from panda3d.bullet import BulletSphereShape
from panda3d.bullet import BulletRigidBodyNode
from panda3d.bullet import BulletHingeConstraint
from panda3d.bullet import BulletDebugNode


class Game(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)
        base.set_background_color(0.1, 0.1, 0.8, 1)
        base.set_frame_rate_meter(True)

        base.cam.set_pos(0, -20, 5)
        base.cam.look_at(0, 0, 0)

        # Light
        alight = AmbientLight('ambientLight')
        alight.set_color((0.5, 0.5, 0.5, 1))
        alightNP = render.attach_new_node(alight)

        dlight = DirectionalLight('directionalLight')
        dlight.set_direction((1, 1, -1))
        dlight.set_color((0.7, 0.7, 0.7, 1))
        dlightNP = render.attach_new_node(dlight)

        render.clear_light()
        render.set_light(alightNP)
        render.set_light(dlightNP)

        # Input
        self.accept('escape', self.do_exit)
        self.accept('r', self.do_reset)
        self.accept('f1', base.toggle_wireframe)
        self.accept('f2', base.toggle_texture)
        self.accept('f3', self.toggle_debug)
        self.accept('f5', self.do_screenshot)

        self.accept('enter', self.do_shoot)

        # Task
        taskMgr.add(self.update, 'updateWorld')

        # Physics
        self.setup()

    def do_exit(self):
        self.cleanup()
        sys.exit(1)

    def do_reset(self):
        self.cleanup()
        self.setup()

    def toggle_debug(self):
        if self.debugNP.is_hidden():
            self.debugNP.show()
        else:
            self.debugNP.hide()

    def do_screenshot(self):
        base.screenshot('Bullet')

    def do_shoot(self):
        # Get from/to points from mouse click
        pMouse = base.mouseWatcherNode.get_mouse()
        pFrom = LPoint3()
        pTo = LPoint3()
        base.camLens.extrude(pMouse, pFrom, pTo)

        pFrom = render.get_relative_point(base.cam, pFrom)
        pTo = render.get_relative_point(base.cam, pTo)

        # Calculate initial velocity
        v = pTo - pFrom
        v.normalize()
        v *= 100.0

        # Create bullet
        shape = BulletSphereShape(0.3)
        body = BulletRigidBodyNode('Bullet')
        bodyNP = self.worldNP.attach_new_node(body)
        bodyNP.node().add_shape(shape)
        bodyNP.node().set_mass(1.0)
        bodyNP.node().set_linear_velocity(v)
        bodyNP.node().set_ccd_motion_threshold(1e-7);
        bodyNP.node().set_ccd_swept_sphere_radius(0.50);
        bodyNP.set_collide_mask(BitMask32.all_on())
        bodyNP.set_pos(pFrom)

        visNP = loader.load_model('models/ball.egg')
        visNP.set_scale(0.8)
        visNP.reparent_to(bodyNP)

        self.world.attach(bodyNP.node())

        # Remove the bullet again after 2 seconds
        taskMgr.do_method_later(2, self.do_remove, 'doRemove',
            extraArgs=[bodyNP], appendTask=True)

    def do_remove(self, bodyNP, task):
        self.world.remove(bodyNP.node())
        bodyNP.remove_node()
        return task.done

    def update(self, task):
        dt = globalClock.get_dt()
        self.world.do_physics(dt, 20, 1.0/180.0)
        return task.cont

    def cleanup(self):
        self.worldNP.remove_node()
        self.worldNP = None
        self.world = None

    def setup(self):
        self.worldNP = render.attach_new_node('World')

        # World
        self.debugNP = self.worldNP.attach_new_node(BulletDebugNode('Debug'))
        self.debugNP.show()
        self.debugNP.node().show_wireframe(True)
        self.debugNP.node().show_constraints(True)
        self.debugNP.node().show_bounding_boxes(False)
        self.debugNP.node().show_normals(False)

        self.world = BulletWorld()
        self.world.set_gravity((0, 0, -9.81))
        self.world.set_debug_node(self.debugNP.node())

        # Box A
        shape = BulletBoxShape((0.5, 0.5, 0.5))

        bodyA = BulletRigidBodyNode('Box A')
        bodyNP = self.worldNP.attach_new_node(bodyA)
        bodyNP.node().add_shape(shape)
        bodyNP.set_collide_mask(BitMask32.all_on())
        bodyNP.set_pos(-2, 0, 1)

        visNP = loader.load_model('models/box.egg')
        visNP.clear_model_nodes()
        visNP.reparent_to(bodyNP)

        self.world.attach(bodyA)

        # Box B
        shape = BulletBoxShape((0.5, 0.5, 0.5))

        bodyB = BulletRigidBodyNode('Box B')
        bodyNP = self.worldNP.attach_new_node(bodyB)
        bodyNP.node().add_shape(shape)
        bodyNP.node().set_mass(1.0)
        bodyNP.node().set_deactivation_enabled(False)
        bodyNP.set_collide_mask(BitMask32.all_on())
        bodyNP.set_pos(2, 0, 1)

        visNP = loader.load_model('models/box.egg')
        visNP.clear_model_nodes()
        visNP.reparent_to(bodyNP)

        self.world.attach(bodyB)

        # Hinge
        pivotA = (2, 0, 0)
        pivotB = (-4, 0, 0)
        axisA = (0, 0, 1)
        axisB = (0, 0, 1)

        hinge = BulletHingeConstraint(bodyA, bodyB, pivotA, pivotB, axisA, axisB, True)
        hinge.set_debug_draw_size(2.0)
        hinge.set_limit(-90, 120, softness=0.9, bias=0.3, relaxation=1.0)
        self.world.attach(hinge)


game = Game()
game.run()

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:loader(error): Couldn't load file models/ball.egg: not found on model path (currently: "/media/ywatcher/ExtDisk1/Files/Games/learn_panda3d/p1:/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/panda3d/etc/..:/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/panda3d/etc/../models")


OSError: Could not load model file(s): ['models/ball.egg']

:task(error): Exception occurred in PythonTask eventManager


OSError: Could not load model file(s): ['models/ball.egg']